# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline , FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
conn = engine.connect()
df = pd.read_sql_table('disaster_messages', conn)
#df = pd.DataFrame(engine.connect().execute(text('SELECT * FROM disaster_messages')))
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    #Remove stop words to reduce training time
    #In fact this improves sensitivity of the model as i found out in the process
    clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
# train pipeline
pipeline.fit(X_train, Y_train)
# predict on test data
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred_mod = pd.DataFrame(Y_pred, columns = Y.columns)

In [7]:
for col in Y_pred_mod.columns:
    print(col)
    print()
    print(classification_report(Y_test[col], Y_pred_mod[col], zero_division=0.0))

related

              precision    recall  f1-score   support

           0       0.71      0.35      0.47      1490
           1       0.82      0.96      0.89      5006
           2       0.47      0.12      0.19        58

    accuracy                           0.81      6554
   macro avg       0.67      0.48      0.52      6554
weighted avg       0.80      0.81      0.78      6554

request

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5439
           1       0.86      0.49      0.62      1115

    accuracy                           0.90      6554
   macro avg       0.88      0.73      0.78      6554
weighted avg       0.90      0.90      0.89      6554

offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6420
           1       0.77      0.13      0.22       134

    accuracy                           0.98      6554
   macro avg       0.88      0.56      0.60      6554
weighted avg       0.98      0.98      0.97      6554

other_weather

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6224
           1       0.64      0.02      0.04       330

    accuracy                           0.95      6554
   macro avg       0.79      0.51      0.51      6554
weighted avg       0.93      0.95      0.93      6554

direct_report

              precision    recall  f1-score   support

           0       0.87      0.98      0.92      5336
           1       0.80      0.36      0.50      1218

    accuracy                           0.86      6554
   macro avg       0.84      0.67      0.71      6554
weighted avg       0.86      0.86      0.84 

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect', CountVectorizer(token_pattern=None,
                   tokenizer=<function tokenize at 0x165c67910>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(token_pattern=None,
                 tokenizer=<function tokenize at 0x165c67910>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': None,
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth

In [9]:
# specify parameters for grid search
parameters = {
    'clf__estimator__class_weight': ['balanced', 'balanced_subsample', None],

}
# Set cv=3 to shorten computing time
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3)

I chose to only modify one parameter here because  it took a long time to calculate for more parameters. Also i believe to have realised a problem with GridSearchCV in combination with MultiOutputClassifier: GridSearchCV needs to evaluate the paramater combination by certain scorings. The optimal scoring to decide on is relatively difficult  to find because in MultiOutput you have different scores for each category. Also, i don't know yet which of the 36 scores exactly GridSearchCV takes to evaluate parameters.

In [10]:
cv.fit(X_train, Y_train)
Y_pred_cv = cv.predict(X_test)

In [11]:
#Try other parameters and RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    'tfidf__norm' : ['l1', 'l2'],
    'tfidf__smooth_idf': [True, False],
    'tfidf__sublinear_tf': [True, False],
    'tfidf__use_idf': [True, False],
    'clf__estimator__max_depth': [3, 5, 10],
    'clf__estimator__n_estimators': [50, 100, 500],
    'clf__estimator__verbose': [0, 5, 20]
}

cv1 = RandomizedSearchCV(pipeline, param_distributions=parameters, cv=3)

In [12]:
cv1.fit(X_train, Y_train)
Y_pred_cv1 = cv1.predict(X_test)

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.1s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100

...

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  50 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  51 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  52 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  53 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  54 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  55 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  57 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  58 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  59 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  60 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  61 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  62 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  63 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  64 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Do

building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91

[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   7 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  11 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  13 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  15 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Do

[Parallel(n_jobs=1)]: Done  79 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  81 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  82 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  83 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  84 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  85 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  86 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  87 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  88 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  89 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  90 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  91 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  92 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  93 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  94 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Do

[Parallel(n_jobs=1)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  35 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  39 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  41 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  43 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  45 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  47 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Do

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
Y_pred_cv_mod = pd.DataFrame(Y_pred_cv)
Y_pred_cv_mod.columns = Y_test.columns

for col in Y_pred_cv_mod.columns:
    print(col)
    print()
    print(classification_report(Y_test[col], Y_pred_cv_mod[col], zero_division=0.0))
    
print("\nBest Parameters:", cv.best_params_)

related

              precision    recall  f1-score   support

           0       0.71      0.35      0.47      1490
           1       0.82      0.96      0.89      5006
           2       0.62      0.09      0.15        58

    accuracy                           0.81      6554
   macro avg       0.72      0.46      0.50      6554
weighted avg       0.80      0.81      0.78      6554

request

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5439
           1       0.86      0.49      0.62      1115

    accuracy                           0.90      6554
   macro avg       0.88      0.74      0.78      6554
weighted avg       0.90      0.90      0.89      6554

offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      

In [14]:
Y_pred_cv1_mod = pd.DataFrame(Y_pred_cv1)
Y_pred_cv1_mod.columns = Y_test.columns

for col in Y_pred_cv1_mod.columns:
    print(col)
    print()
    print(classification_report(Y_test[col], Y_pred_cv1_mod[col], zero_division=0.0))
    
print("\nBest Parameters:", cv1.best_params_)

related

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1490
           1       0.76      1.00      0.87      5006
           2       0.00      0.00      0.00        58

    accuracy                           0.76      6554
   macro avg       0.25      0.33      0.29      6554
weighted avg       0.58      0.76      0.66      6554

request

              precision    recall  f1-score   support

           0       0.83      1.00      0.91      5439
           1       0.00      0.00      0.00      1115

    accuracy                           0.83      6554
   macro avg       0.41      0.50      0.45      6554
weighted avg       0.69      0.83      0.75      6554

offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      

Baseline: GridSearchCV and RandomizedSearchCV did not lead to better results. In fact, sensitivity of the model was totally killed. Both searches lead to only zeros. It seems i did not choose the best combinations of Hyperparameters to dive into. Also maybe the scoring parameters lead the model to choosing not really the best parameter. For now it's too much for me to handle alone. If to dive deep into readings, which i will, to improve parameter settings like scoring and hyperparameter settings.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

I want to try and include the clustering feature KMeans.

In [34]:
from sklearn.cluster import KMeans
pipeline_new = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)),
    ('tfidf', TfidfTransformer()),
    ('kmeans', KMeans()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [16]:
# train pipeline
pipeline_new.fit(X_train, Y_train)
# predict on test data
Y_pred = pipeline_new.predict(X_test)

In [17]:
Y_pred_mod = pd.DataFrame(Y_pred, columns=Y_test.columns)

In [18]:
for col in Y_pred_mod.columns:
    print(col)
    print()
    print(classification_report(Y_test[col], Y_pred_mod[col], zero_division=0.0))

related

              precision    recall  f1-score   support

           0       0.59      0.22      0.32      1490
           1       0.80      0.96      0.87      5006
           2       0.67      0.07      0.12        58

    accuracy                           0.78      6554
   macro avg       0.69      0.42      0.44      6554
weighted avg       0.75      0.78      0.74      6554

request

              precision    recall  f1-score   support

           0       0.90      0.96      0.93      5439
           1       0.69      0.48      0.56      1115

    accuracy                           0.87      6554
   macro avg       0.79      0.72      0.74      6554
weighted avg       0.86      0.87      0.86      6554

offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      

              precision    recall  f1-score   support

           0       0.87      0.95      0.91      5336
           1       0.65      0.40      0.50      1218

    accuracy                           0.85      6554
   macro avg       0.76      0.68      0.70      6554
weighted avg       0.83      0.85      0.83      6554



The above results are somewhat worse then the original results. Maybe implementing the feature one after the other did weaken the effects of tfidf. Let's try a FeatureUnion of tfidf and KMeans.

In [30]:
pipeline_new = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)),
    ('feature_union', FeatureUnion([
        ('tfidf', TfidfTransformer()),
        ('kmeans', KMeans()),
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [31]:
# train pipeline
pipeline_new.fit(X_train, Y_train)
# predict on test data
Y_pred = pipeline_new.predict(X_test)

In [32]:
Y_pred_mod = pd.DataFrame(Y_pred, columns=Y_test.columns)

In [33]:
for col in Y_pred_mod.columns:
    print(col)
    print()
    print(classification_report(Y_test[col], Y_pred_mod[col], zero_division=0.0))

related

              precision    recall  f1-score   support

           0       0.71      0.30      0.42      1490
           1       0.82      0.96      0.88      5006
           2       0.75      0.05      0.10        58

    accuracy                           0.81      6554
   macro avg       0.76      0.44      0.47      6554
weighted avg       0.79      0.81      0.77      6554

request

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5439
           1       0.84      0.46      0.59      1115

    accuracy                           0.89      6554
   macro avg       0.87      0.72      0.77      6554
weighted avg       0.89      0.89      0.88      6554

offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      

That's not a success. The precision and recall values are in fact a little lower than in the original results.
Let's export our original 'pipeline' then.

### 9. Export your model as a pickle file

In [35]:
import pickle

pickle.dump(pipeline, open('disaster-response-model.pk1' , 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.